[张量程序抽象](#张量程序抽象)

[TensorIR：张量程序抽象案例研究](#TensorIR：张量程序抽象案例研究)

[转换张量程序](#转换张量程序)

[构建和运行](#构建和运行)

[创建TensorIR并与之交互的方法](#创建tensorir并与之交互的方法)


In [1]:
import tvm
from tvm.ir.module import IRModule
from tvm.script import tir as T
import numpy as np


 ## 元张量函数
*****
- MLC的过程可以看作张量函数间的变换。

- <font color=red>元张量函数</font>
    - 机器学习模型的执行包含将输入张量转换到输出预测的多步计算，其中的每一步都为元张量函数。
    - 同一个元张量函数可以由多种不同的抽象表示和实现。
        - 如：调用预编译框架库(torch.add numpy.add)
        - 目标：将元张量函数转换为更专门和针对特定工作和部署环境的函数。

![元张量函数间的变换](img/tensor_func_transformation.png)
- 左图中的add被转换为右侧的一段表示可能的组合优化的add(被拆分为长度为4的单元)

<span id="张量程序抽象"></span>

# 张量程序抽象
- 针对元张量函数变换的需要，需要一个有效的抽象来表示，从而有效地变换元张量函数
- 一个典型抽象包含：
    - 存储数据的多维数组
    - 驱动张量计算的循环嵌套
    - 计算部分语句
- 张量程序抽象的一个重要性质：能够被一系列有效的程序变换所改变
*****
- 我们不能随意对程序进行变换，因为一些计算会依赖循环间的顺序。
- 大多数元张量函数都有良好的属性，如循环迭代间的独立性。

下图中包含额外的T.axis.spatial标注，表明 vi 这个特定的变量被映射到循环变量 i，并且所有的迭代都是独立的。
- 这个信息会使得我们在变换这个程序时更加方便。我们可以安全地并行或者重新排序所有与 vi 有关的循环，只要实际执行中 vi 的值按照从 0 到 128 的顺序变化。


In [4]:
@tvm.script.ir_module
class MyModule:

    # 张量程序抽象
    @T.prim_func
    def main(A: T.Buffer[128, "float32"],   # 多维张量缓冲区：用来存放输入，输出，中间结果
             B: T.Buffer[128, "float32"], 
             C: T.Buffer[128, "float32"]):
        # extra annotations for the function
        T.func_attr({"global_symbol": "main", "tir.noalias": True})
        for i in range(128):                # 驱动张量计算的循环嵌套
            with T.block("C"):
                # declare a data parallel iterator on spatial domain
                vi = T.axis.spatial(128, i)
                C[vi] = A[vi] + B[vi]       # 计算语句

# TVMScript 用于在python ast中表达张量程序。
# 虽然这段代码不等价于python程序，但可以在MLC过程中使用。
# 该语言旨在与python语法一致，且有便于分析和转换的额外结构。

type(MyModule)
# MyMudule的数据结构为IRModule：用于保存一群张量函数的集合
# 这一数据结构可以用script函数获取字符串形式的表达，这是一个用于检查每步转换后模块的重要方法。
print(MyModule.script())

# from tvm.script import tir as T
@tvm.script.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer[128, "float32"], B: T.Buffer[128, "float32"], C: T.Buffer[128, "float32"]) -> None:
        # function attr dict
        T.func_attr({"global_symbol": "main", "tir.noalias": True})
        # body
        # with T.block("root")
        for i in T.serial(128):
            with T.block("C"):
                vi = T.axis.spatial(128, i)
                T.reads(A[vi], B[vi])
                T.writes(C[vi])
                C[vi] = A[vi] + B[vi]
    



- 通过tvm.build将IRModule转化为可运行函数
- build后的mod包含一群可运行函数的集合，可以通过函数名取回函数

In [6]:
rt_mod = tvm.build(MyModule, target="llvm")  # The module for CPU backends.
print(type(rt_mod))

func = rt_mod["main"]
print(func)


<class 'tvm.driver.build_module.OperatorModule'>


In [ ]:
# 在tvm runtime创建三个NDArray
a = tvm.nd.array(np.arange(128, dtype="float32"))
b = tvm.nd.array(np.ones(128, dtype="float32"))
c = tvm.nd.empty((128,), dtype="float32") 
func(a,b,c)
print(a)
print(b)
print(c)

In [ ]:
# 分解循环
sch = tvm.tir.Schedule(MyModule)
print(type(sch))

# Get block by its name
block_c = sch.get_block("C")
# Get loops surronding the block
(i,) = sch.get_loops(block_c)
# Tile the loop nesting.
i_0, i_1, i_2 = sch.split(i, factors=[None, 4, 4])
print(sch.mod.script())

<span id="TensorIR：张量程序抽象案例研究"></span>

# TensorIR：张量程序抽象案例研究
TensorIR 是标准机器学习编译框架 Apache TVM 中使用的张量程序抽象。

使用张量程序抽象的主要目的是表示循环和相关的硬件加速选择，如多线程、特殊硬件指令的使用和内存访问。

以两个128×128大小的矩阵A和B进行两步计算： matmul->relu 线性层和激活层

In [7]:
import numpy as np
import tvm
from tvm.ir.module import IRModule
from tvm.script import tir as T

## 简单的numpy实现

在底层，NumPy 调用库（例如 OpenBLAS）和它自己在低级 C 语言中的一些实现来执行这些计算。

In [9]:
dtype = "float32"
a_np = np.random.rand(128, 128).astype(dtype)
b_np = np.random.rand(128, 128).astype(dtype)
# a @ b is equivalent to np.matmul(a, b)
c_mm_relu = np.maximum(a_np @ b_np, 0)

底层细节实现

为了说明底层细节，我们将在 NumPy API 的一个受限子集中编写示例 —— 我们称之为 低级 NumPy。它使用以下的约定：

- 我们将在必要时使用循环而不是数组函数来展示可能的循环计算。
- 如果可能，我们总是通过 numpy.empty 显式地分配数组并传递它们。

需要注意的是，这不是人们通常编写 NumPy 程序的方式。不过，它们仍然与幕后发生的事情非常相似 —— 大多数现实世界的部署解决方案都将分配与计算分开处理。特定的库使用不同形式的循环和算术计算来执行计算。当然首先它们是使用诸如 C 之类的低级语言实现的。

In [ ]:
def lnumpy_mm_relu(A: np.ndarray, B: np.ndarray, C: np.ndarray):
    # 首先分配中间存储A，存放矩阵乘结果
    Y = np.empty((128, 128), dtype="float32")
    for i in range(128):
        for j in range(128):
            for k in range(128):
                if k == 0:
                    Y[i, j] = 0
                Y[i, j] = Y[i, j] + A[i, k] * B[k, j]
    # 计算RELU
    for i in range(128):
        for j in range(128):
            C[i, j] = max(Y[i, j], 0)

c_np = np.empty((128, 128), dtype=dtype)
lnumpy_mm_relu(a_np, b_np, c_np)
# 对比验证结果正确性
np.testing.assert_allclose(c_mm_relu, c_np, rtol=1e-5)

## TensorIR
使用TVMScript语言实现

- 主要区别
    - 计算块中T.block额外结构。块是TensoIR中的基本计算单位。
        - 一个块包含一组块轴(vi, vj, vk)和围绕他们定义的计算。
            - 块轴的<font color=red>关键性质</font>用T.axis.[axis_type]声明
            - 语法：[block_axis] = T.axis.[axis_type]([axis_range], [mapped_value])     
                - [axis_range]提供了[block_axis]的预期范围 如vi = T.axis.spatial(128.i)表面vi值应该在(0,128)中。
            

In [3]:
@tvm.script.ir_module
class MyModule:
    @T.prim_func
    def mm_relu(A: T.Buffer[(128, 128), "float32"],
                B: T.Buffer[(128, 128), "float32"],
                C: T.Buffer[(128, 128), "float32"]):
        
        T.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        # 首先分配中间存储A，存放矩阵乘结果
        Y = T.alloc_buffer((128, 128), dtype="float32")
        # 循环迭代
        for i, j, k in T.grid(128, 128, 128):
            # 计算块
            with T.block("Y"):
                vi = T.axis.spatial(128, i)
                vj = T.axis.spatial(128, j)
                vk = T.axis.reduce(128, k)
                with T.init():
                    Y[vi, vj] = T.float32(0)
                Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
        for i, j in T.grid(128, 128):
            with T.block("C"):
                vi = T.axis.spatial(128, i)
                vj = T.axis.spatial(128, j)
                C[vi, vj] = T.max(Y[vi, vj], T.float32(0))

### 块轴的属性

这些轴属性标记了轴与正在执行的计算之间的关系。

下图总结了块（迭代）轴和块 Y 的读写关系。注意到，严格来说这个块正在对缓冲区 Y 进行（规约）更新，我们暂时将其标记为写入，因为我们不需要来自另一个块的缓冲区 Y 的值。

![块轴与计算的关系](img/tensor_ir_block_axis.png)

块 Y 通过读取来自 A[vi, vk] 和 B[vk, vj] 的值来计算结果 Y[vi, vj]，并对所有可能的 vk 执行求和。 在这个特定示例中，如果我们将 vi、vj 固定为 (0, 1)，并对 vk in range(0, 128) 执行块 Y，我们可以独立于其他可能的位置（具有不同 vi, vj 值的位置）有效地计算 C[0, 1]。

值得注意的是，对于一组固定的 vi 和 vj，计算块在 Y 的空间位置 (Y[vi, vj]) 处生成一个点值，该点值独立于 Y 中的其他位置（具有不同的vi, vj 值的位置）。我们可以称 vi、vj 为<font color=red>空间轴</font>，因为它们直接对应于块写入的缓冲区空间区域的开始。 涉及归约（vk）的轴被命名为<font color=red>归约轴</font>。

- 这些附加信息有助于我们进行机器学习编译分析。例如，虽然我们总是可以在空间轴上做并行化，在规约轴上进行并行化将需要特定的策略。

块轴绑定的语法糖

- 在每个块轴直接映射到外部循环迭代器的情况下，我们可以使用 T.axis.remap 在一行中声明所有块轴。

- SSR means the properties of each axes are "spatial", "spatial", "reduce"

- vi, vj, vk = T.axis.remap("SSR", [i, j, k])

### 函数属性和装饰器
T.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})

- 这里的 global_symbol 对应函数名，tir.noalias 是一个属性，表示所有的缓冲存储器不重叠。你现在可以放心地跳过这些属性，因为它们不会影响对概念的整体理解。

@tvm.script.ir_module 和 @T.prim_func 这两个装饰器用于表示对应部分的类型。

- @tvm.script.ir_module 表示 MyModule 是一个 IRModule。IRModule 是在机器学习编译中保存张量函数集合的容器对象。

机器学习编译过程中的一个 IRModule 可以包含多个张量函数。 以下代码块显示了具有两个函数的 IRModule 示例。

<span id="转换张量程序"></span>

# 转换张量程序
- 用辅助数据结构schedule来转换张量程序。
- 核心问题：
    - 什么是表示张量函数可能的抽象？
    - 什么是张量函数间可能的变换？

In [4]:
import IPython

IPython.display.Code(MyModule.script(), language="python")

# from tvm.script import tir as T
@tvm.script.ir_module
class Module:
    @T.prim_func
    def mm_relu(A: T.Buffer[(128, 128), "float32"], B: T.Buffer[(128, 128), "float32"], C: T.Buffer[(128, 128), "float32"]) -> None:
        # function attr dict
        T.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        # body
        # with T.block("root")
        Y = T.alloc_buffer([128, 128], dtype="float32")
        for i, j, k in T.grid(128, 128, 128):
            with T.block("Y"):
                vi, vj, vk = T.axis.remap("SSR", [i, j, k])
                T.reads(A[vi, vk], B[vk, vj])
                T.writes(Y[vi, vj])
                with T.init():
                    Y[vi, vj] = T.float32(0)
                Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
        for i, j in T.grid(128, 128):
            with T.block("C"):
                vi, vj = T.axis.remap("SS", [i, j])
                T.reads(Y[vi, vj])
                T.writes(C[vi, vj])
                C[vi, vj] = T.max(Y[vi, vj], T.float32(0))

In [6]:
# 首先创建一个以给定的 MyModule 作为输入的 Schedule 辅助类。
sch = tvm.tir.Schedule(MyModule)
# 获得对块 Y 和相应循环的引用
block_Y = sch.get_block("Y", func_name="mm_relu")
i, j, k = sch.get_loops(block_Y)

#将执行的第一个变换是将循环 j 分成两个循环，其中内部循环的长度为 4。
j0, j1 = sch.split(j, factors=[None, 4])
#查看存储在 sch.mod 中的变换结果。
IPython.display.Code(sch.mod.script(), language="python")

# from tvm.script import tir as T
@tvm.script.ir_module
class Module:
    @T.prim_func
    def mm_relu(A: T.Buffer[(128, 128), "float32"], B: T.Buffer[(128, 128), "float32"], C: T.Buffer[(128, 128), "float32"]) -> None:
        # function attr dict
        T.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        # body
        # with T.block("root")
        Y = T.alloc_buffer([128, 128], dtype="float32")
        for i, j_0, j_1, k in T.grid(128, 32, 4, 128):
            with T.block("Y"):
                vi = T.axis.spatial(128, i)
                vj = T.axis.spatial(128, j_0 * 4 + j_1)
                vk = T.axis.reduce(128, k)
                T.reads(A[vi, vk], B[vk, vj])
                T.writes(Y[vi, vj])
                with T.init():
                    Y[vi, vj] = T.float32(0)
                Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
        for i, j in T.grid(128, 128):
            with T.block("C"):
                vi, vj = T.axis.remap("SS", [i, j])
                T.reads(Y[vi, vj])
                T.writes(C[vi, vj])
                C[vi, vj] = T.max(Y[vi, vj], T.float32(0))

In [7]:
# 重新排序这两个循环。
sch.reorder(j0, k, j1)
IPython.display.Code(sch.mod.script(), language="python")

# from tvm.script import tir as T
@tvm.script.ir_module
class Module:
    @T.prim_func
    def mm_relu(A: T.Buffer[(128, 128), "float32"], B: T.Buffer[(128, 128), "float32"], C: T.Buffer[(128, 128), "float32"]) -> None:
        # function attr dict
        T.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        # body
        # with T.block("root")
        Y = T.alloc_buffer([128, 128], dtype="float32")
        for i, j_0, k, j_1 in T.grid(128, 32, 128, 4):
            with T.block("Y"):
                vi = T.axis.spatial(128, i)
                vj = T.axis.spatial(128, j_0 * 4 + j_1)
                vk = T.axis.reduce(128, k)
                T.reads(A[vi, vk], B[vk, vj])
                T.writes(Y[vi, vj])
                with T.init():
                    Y[vi, vj] = T.float32(0)
                Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
        for i, j in T.grid(128, 128):
            with T.block("C"):
                vi, vj = T.axis.remap("SS", [i, j])
                T.reads(Y[vi, vj])
                T.writes(C[vi, vj])
                C[vi, vj] = T.max(Y[vi, vj], T.float32(0))

## 另一个变体
将块 C 移动到 Y 的内循环里

将归约初始化和更新放在一个块体中。这种组合形式为循环变换带来了便利（因为初始化和更新的外循环 i、j 通常需要彼此保持同步）。

In [8]:
block_C = sch.get_block("C", "mm_relu")
sch.reverse_compute_at(block_C, j0)
IPython.display.Code(sch.mod.script(), language="python")

# from tvm.script import tir as T
@tvm.script.ir_module
class Module:
    @T.prim_func
    def mm_relu(A: T.Buffer[(128, 128), "float32"], B: T.Buffer[(128, 128), "float32"], C: T.Buffer[(128, 128), "float32"]) -> None:
        # function attr dict
        T.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        # body
        # with T.block("root")
        Y = T.alloc_buffer([128, 128], dtype="float32")
        for i, j_0 in T.grid(128, 32):
            for k, j_1 in T.grid(128, 4):
                with T.block("Y"):
                    vi = T.axis.spatial(128, i)
                    vj = T.axis.spatial(128, j_0 * 4 + j_1)
                    vk = T.axis.reduce(128, k)
                    T.reads(A[vi, vk], B[vk, vj])
                    T.writes(Y[vi, vj])
                    with T.init():
                        Y[vi, vj] = T.float32(0)
                    Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
            for ax0 in T.serial(4):
                with T.block("C"):
                    vi = T.axis.spatial(128, i)
                    vj = T.axis.spatial(128, j_0 * 4 + ax0)
                    T.reads(Y[vi, vj])
                    T.writes(C[vi, vj])
                    C[vi, vj] = T.max(Y[vi, vj], T.float32(0))

在循环变换之后，我们可以将 Y 元素的初始化与归约更新分开。我们可以通过 decompose_reduction 原语来做到这一点。（注意：这也是 TVM 在以后编译的时候隐式做的，所以这一步的主要目的是让它显式，看看最终效果）。

In [9]:
sch.decompose_reduction(block_Y, k)
IPython.display.Code(sch.mod.script(), language="python")

# from tvm.script import tir as T
@tvm.script.ir_module
class Module:
    @T.prim_func
    def mm_relu(A: T.Buffer[(128, 128), "float32"], B: T.Buffer[(128, 128), "float32"], C: T.Buffer[(128, 128), "float32"]) -> None:
        # function attr dict
        T.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        # body
        # with T.block("root")
        Y = T.alloc_buffer([128, 128], dtype="float32")
        for i, j_0 in T.grid(128, 32):
            for j_1_init in T.serial(4):
                with T.block("Y_init"):
                    vi = T.axis.spatial(128, i)
                    vj = T.axis.spatial(128, j_0 * 4 + j_1_init)
                    T.reads()
                    T.writes(Y[vi, vj])
                    Y[vi, vj] = T.float32(0)
            for k, j_1 in T.grid(128, 4):
                with T.block("Y_update"):
                    vi = T.axis.spatial(128, i)
                    vj = T.axis.spatial(128, j_0 * 4 + j_1)
                    vk = T.axis.reduce(128, k)
                    T.reads(Y[vi, vj], A[vi, vk], B[vk, vj])
                    T.writes(Y[vi, vj])
                    Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
            for ax0 in T.serial(4):
                with T.block("C"):
                    vi = T.axis.spatial(128, i)
                    vj = T.axis.spatial(128, j_0 * 4 + ax0)
                    T.reads(Y[vi, vj])
                    T.writes(C[vi, vj])
                    C[vi, vj] = T.max(Y[vi, vj], T.float32(0))

<span id="构建和运行"></span>

# 构建和运行


In [19]:
# 原始
rt_lib = tvm.build(MyModule, target="llvm")
a_nd = tvm.nd.array(a_np)
b_nd = tvm.nd.array(b_np)
c_nd = tvm.nd.empty((128, 128), dtype="float32")
func_mm_relu = rt_lib["mm_relu"]
func_mm_relu(a_nd, b_nd, c_nd)

np.testing.assert_allclose(c_mm_relu, c_nd.numpy(), rtol=1e-5)

In [20]:
# 变换后
rt_lib_after = tvm.build(sch.mod, target="llvm")
rt_lib_after["mm_relu"](a_nd, b_nd, c_nd)
np.testing.assert_allclose(c_mm_relu, c_nd.numpy(), rtol=1e-5)

In [21]:
# 时间差
f_timer_before = rt_lib.time_evaluator("mm_relu", tvm.cpu())
print("Time cost of MyModule %g sec" % f_timer_before(a_nd, b_nd, c_nd).mean)
f_timer_after = rt_lib_after.time_evaluator("mm_relu", tvm.cpu())
print("Time cost of transformed sch.mod %g sec" % f_timer_after(a_nd, b_nd, c_nd).mean)

Time cost of MyModule 0.00308494 sec
Time cost of transformed sch.mod 0.000684898 sec


- 分析影响性能的可能因素
![CPU架构](img/cpu_arch.png)

    - 访问 A 和 B 中的任何内存块的速度并不一致。现代 CPU 带有多级缓存，需要先将数据提取到缓存中，然后 CPU 才能访问它。
    - 重要的是，访问已经在缓存中的数据要快得多。CPU 采用的一种策略是获取彼此更接近的数据。 当我们读取内存中的一个元素时，它会尝试将附近的元素（更正式的名称为“缓存行”）获取到缓存中。 因此，当你读取下一个元素时，它已经在缓存中。 因此，具有连续内存访问的代码通常比随机访问内存不同部分的代码更快。

![可视化](img/tensor_func_loop_order.png)
现在让我们看看上面的迭代可视化，分析一下是怎么回事。 在这个分析中，让我们关注最里面的两个循环：k 和 j1。高亮的地方显示了当我们针对 k 的一个特定实例迭代 j1 时迭代触及的 Y、A 和 B 中的相应区域。

我们可以发现，j1 这一迭代产生了对 B 元素的连续访问。具体来说，它意味着在 j1=0 和 j1=1 时我们读取的值彼此相邻。这可以让我们拥有更好的缓存访问行为。此外，我们使 C 的计算更接近 Y，从而实现更好的缓存行为。

我们当前的示例主要是为了证明不同的代码变体可以导致不同的性能。更多的变换步骤可以帮助我们获得更好的性能，我们将在以后的章节中介绍。本练习的主要目标是首先让我们获得程序变换工具，并首先体验通过变换可能实现的功能。



<span id="创建TensorIR并与之交互的方法"></span>

# 创建TensorIR并与之交互的方法

- 用TVMScript的方式
- 用张量表达式TE生成TensorIR代码
    - 帮助为给定的更高级别的输入生成TensorIR函数


In [11]:
from tvm import te

A = te.placeholder((128, 128), "float32", name="A")
B = te.placeholder((128, 128), "float32", name="B")
k = te.reduce_axis((0, 128), "k")
Y = te.compute((128, 128), lambda i, j: te.sum(A[i, k] * B[k, j], axis=k), name="Y")
C = te.compute((128, 128), lambda i, j: te.max(Y[i, j], 0), name="C")

# 由上述TE表达式转换为TensorIR
te_func = te.create_prim_func([A, B, C]).with_attr({"global_symbol": "mm_relu"})
MyModuleFromTE = tvm.IRModule({"mm_relu": te_func})
IPython.display.Code(MyModuleFromTE.script(), language="python")



# from tvm.script import tir as T
@tvm.script.ir_module
class Module:
    @T.prim_func
    def mm_relu(A: T.Buffer[(128, 128), "float32"], B: T.Buffer[(128, 128), "float32"], C: T.Buffer[(128, 128), "float32"]) -> None:
        # function attr dict
        T.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        # body
        # with T.block("root")
        Y = T.alloc_buffer([128, 128], dtype="float32")
        for i0, i1, i2 in T.grid(128, 128, 128):
            with T.block("Y"):
                i, j, k = T.axis.remap("SSR", [i0, i1, i2])
                T.reads(A[i, k], B[k, j])
                T.writes(Y[i, j])
                with T.init():
                    Y[i, j] = T.float32(0)
                Y[i, j] = Y[i, j] + A[i, k] * B[k, j]
        for i0, i1 in T.grid(128, 128):
            with T.block("C"):
                i, j = T.axis.remap("SS", [i0, i1])
                T.reads(Y[i, j])
                T.writes(C[i, j])
                C[i, j] = T.max(Y[i, j], T.float32(0))

MLC的开发过程
![mlc_process](img/mlc_process.png)